In [7]:


import cv2
from ultralytics import YOLO
import supervision as sv
import time

# Load the video
video_path = "test.mp4"
cap = cv2.VideoCapture(video_path)

# Get FPS of video
FPS = cap.get(cv2.CAP_PROP_FPS)  

# Load YOLOv8 for vehicle detection
model = YOLO("yolov8n.pt")  

tracker = sv.ByteTrack()  # max_age ensures smooth tracking over frames

# Calibration: Define real-world meters per pixel (adjust based on your scene)
METERS_PER_PIXEL = 0.02  

# Dictionary to store previous positions of vehicles
previous_positions = {}


In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    start_time = time.time()

    # Perform detection with YOLO
    results = model(frame)

    detected_objects = []
    
    # Process results
    def callback(frame: np.ndarray, _: int) -> np.ndarray:
        results = model(frame)[0]
        detections = sv.Detections.from_ultralytics(results)
        detections = tracker.update_with_detections(detections)
        return box_annotator.annotate(frame.copy(), detections=detections)

    # Update tracker
    trackers = tracker.update_with_detections(detections=detected_objects)

    # Annotate and track objects
    for track in trackers:
        track_id = track[4]  # Unique ID assigned to each tracked object
        x1, y1, x2, y2 = map(int, track[:4])
        center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2

        # Speed calculation (based on centroid movement)
        if track_id in previous_positions:
            prev_x, prev_y, prev_time = previous_positions[track_id]
            pixel_distance = ((center_x - prev_x) ** 2 + (center_y - prev_y) ** 2) ** 0.5
            distance_meters = pixel_distance * METERS_PER_PIXEL
            elapsed_time = time.time() - prev_time

            if elapsed_time > 0:
                speed_mps = distance_meters / elapsed_time
                speed_kmh = speed_mps * 3.6
                cv2.putText(frame, f"{int(speed_kmh)} km/h", (x1, y1 - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        previous_positions[track_id] = (center_x, center_y, time.time())

        # Draw bounding box and ID
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID {track_id}", (x1 + 10, y1 - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    # Display the annotated frame
    cv2.imshow("Vehicle Speed Detection", frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Clean up
cap.release()
cv2.destroyAllWindows()


0: 384x640 3 cars, 245.8ms
Speed: 17.0ms preprocess, 245.8ms inference, 4.1ms postprocess per image at shape (1, 3, 384, 640)


AttributeError: 'list' object has no attribute 'xyxy'

In [10]:
import numpy as np
import supervision as sv
from ultralytics import YOLO
import cv2
import time

# Load YOLO model
model = YOLO("yolov8n.pt")  # Load YOLOv8 model (ensure the model path is correct)
tracker = sv.ByteTrack()  # Using ByteTrack tracker for object tracking
box_annotator = sv.BoundingBoxAnnotator()  # Annotator for bounding boxes
label_annotator = sv.LabelAnnotator()  # Annotator for labels (IDs, class names)
trace_annotator = sv.TraceAnnotator()  # Annotator for tracking traces

# Initialize previous positions dictionary for speed calculation
previous_positions = {}

# Calibration factor (adjust based on your setup)
METERS_PER_PIXEL = 0.02  # Convert pixels to meters

def callback(frame: np.ndarray, _: int) -> np.ndarray:
    # Perform object detection with YOLOv8
    results = model(frame)[0]
    
    # Convert results to Supervision Detections format
    detections = sv.Detections.from_ultralytics(results)
    
    # Update the tracker with new detections
    detections = tracker.update_with_detections(detections)
    
    # Prepare labels (ID and class name)
    labels = [
        f"#{tracker_id} {results.names[class_id]}"
        for class_id, tracker_id
        in zip(detections.class_id, detections.tracker_id)
    ]
    
    # Annotate bounding boxes and labels on the frame
    annotated_frame = box_annotator.annotate(frame.copy(), detections=detections)
    annotated_frame = label_annotator.annotate(annotated_frame, detections=detections, labels=labels)
    
    # Speed calculation and annotation
    for tracker_id, (x1, y1, x2, y2) in zip(detections.tracker_id, detections.xyxy):
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
        center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2

        # Speed calculation based on previous position
        if tracker_id in previous_positions:
            prev_x, prev_y, prev_time = previous_positions[tracker_id]
            pixel_distance = ((center_x - prev_x) ** 2 + (center_y - prev_y) ** 2) ** 0.5
            distance_meters = pixel_distance * METERS_PER_PIXEL
            elapsed_time = time.time() - prev_time

            if elapsed_time > 0:
                speed_mps = distance_meters / elapsed_time
                speed_kmh = speed_mps * 3.6  # Convert from meters per second to kilometers per hour
                cv2.putText(annotated_frame, f"{int(speed_kmh)} km/h", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # Store current position and time for next frame
        previous_positions[tracker_id] = (center_x, center_y, time.time())

    # Annotate the traces of the objects (for visual tracking of movement)
    return trace_annotator.annotate(annotated_frame, detections=detections)

# Process the video using Supervision library
sv.process_video(
    source_path="test.mp4",  # Your input video path
    target_path="result_traffic.mp4",  # Output video path
    callback=callback
)


SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.



0: 384x640 3 cars, 183.6ms
Speed: 9.9ms preprocess, 183.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 198.7ms
Speed: 6.1ms preprocess, 198.7ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 151.0ms
Speed: 8.7ms preprocess, 151.0ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 208.1ms
Speed: 10.2ms preprocess, 208.1ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 175.7ms
Speed: 5.1ms preprocess, 175.7ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 195.4ms
Speed: 3.2ms preprocess, 195.4ms inference, 3.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 173.3ms
Speed: 5.3ms preprocess, 173.3ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 182.8ms
Speed: 7.6ms preprocess, 182.8ms inference, 2.7ms postprocess per image at shape (1, 3, 3

KeyboardInterrupt: 

In [2]:
import numpy as np
import supervision as sv
from ultralytics import YOLO
import cv2
import time

# Load YOLO model
model = YOLO("yolov8n.pt")  # Ensure this path is correct
tracker = sv.ByteTrack()  # Using ByteTrack tracker
box_annotator = sv.BoundingBoxAnnotator()
label_annotator = sv.LabelAnnotator()
trace_annotator = sv.TraceAnnotator()

# Calibration factor (adjust based on your setup)
METERS_PER_PIXEL = 0.02  # Conversion from pixels to meters

# Define two lines (adjust based on the video)
start_line_y = 300  # Start line position (y-coordinate)
end_line_y = 400  # End line position (y-coordinate)

# Initialize previous positions and timestamps
cross_times = {}  # Track car crossing times

# OpenCV VideoCapture for reading the video file
cap = cv2.VideoCapture("test2.mp4")  # Replace with your video file path

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break
    
    # Perform object detection
    results = model(frame)[0]
    
    # Convert results to Supervision Detections format
    detections = sv.Detections.from_ultralytics(results)
    
    # Update tracker with new detections
    detections = tracker.update_with_detections(detections)
    
    # Prepare labels
    labels = [
        f"#{tracker_id} {results.names[class_id]}"
        for class_id, tracker_id in zip(detections.class_id, detections.tracker_id)
    ]
    
    # Annotate bounding boxes and labels
    annotated_frame = box_annotator.annotate(frame.copy(), detections=detections)
    annotated_frame = label_annotator.annotate(annotated_frame, detections=detections, labels=labels)

    # Draw start and end lines on the frame
    cv2.line(annotated_frame, (0, start_line_y), (frame.shape[1], start_line_y), (0, 255, 0), 2)
    cv2.line(annotated_frame, (0, end_line_y), (frame.shape[1], end_line_y), (0, 255, 0), 2)

    # Process each tracked object
    for tracker_id, (x1, y1, x2, y2) in zip(detections.tracker_id, detections.xyxy):
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
        center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2

        if tracker_id not in cross_times:
            cross_times[tracker_id] = {"start": None, "end": None}

        # If car crosses the start line
        if cross_times[tracker_id]["start"] is None and start_line_y - 10 < center_y < start_line_y + 10:
            cross_times[tracker_id]["start"] = time.time()
            print(f"Car {tracker_id} crossed start line.")

        # If car crosses the end line
        if cross_times[tracker_id]["start"] is not None and cross_times[tracker_id]["end"] is None:
            if end_line_y - 10 < center_y < end_line_y + 10:
                cross_times[tracker_id]["end"] = time.time()
                print(f"Car {tracker_id} crossed end line.")

                # Calculate time to cross lines
                start_time = cross_times[tracker_id]["start"]
                end_time = cross_times[tracker_id]["end"]
                time_to_cross = end_time - start_time

                if time_to_cross > 0:
                    # Calculate the distance between the lines (in meters)
                    distance_pixels = abs(end_line_y - start_line_y)
                    distance_meters = distance_pixels * METERS_PER_PIXEL

                    # Calculate speed (in meters per second)
                    speed_mps = distance_meters / time_to_cross
                    speed_kmh = speed_mps * 3.6  # Convert to km/h

                    # Annotate speed on the frame
                    speed_text = f"{int(speed_kmh)} km/h"
                    cv2.putText(annotated_frame, speed_text, (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                    print(f"Car {tracker_id} speed: {int(speed_kmh)} km/h")

        # Reset crossing times once car has passed both lines
        if cross_times[tracker_id]["end"] is not None:
            cross_times[tracker_id]["start"] = None
            cross_times[tracker_id]["end"] = None

    # Annotate object traces
    annotated_frame = trace_annotator.annotate(annotated_frame, detections=detections)
    
    # Show the frame with annotations
    cv2.imshow("Speed Detection", annotated_frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close windows
cap.release()
cv2.destroyAllWindows()


SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.



0: 384x640 8 cars, 1 truck, 711.4ms
Speed: 3.9ms preprocess, 711.4ms inference, 151.2ms postprocess per image at shape (1, 3, 384, 640)
Car 5 crossed start line.

0: 384x640 8 cars, 2 trucks, 111.1ms
Speed: 6.2ms preprocess, 111.1ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 1 truck, 106.5ms
Speed: 5.8ms preprocess, 106.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 2 trucks, 116.8ms
Speed: 3.8ms preprocess, 116.8ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 1 truck, 98.5ms
Speed: 4.3ms preprocess, 98.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 1 truck, 102.8ms
Speed: 5.5ms preprocess, 102.8ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 1 truck, 98.3ms
Speed: 4.4ms preprocess, 98.3ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 1 bus, 1 

KeyboardInterrupt: 